In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', 500)

In [2]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / (1024 ** 2)    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / (1024 ** 2)
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
def binarize(df):
    for col in ('authorized_flag', 'category_1'):
        df[col] = df[col].map({'Y': 1, 'N': 0})
    return df

In [4]:
#np.iinfo?

In [5]:
def read_data(input_file_patch):
    df = pd.read_csv(input_file_patch)
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['elapsed_time'] = (datetime.date(2018, 2, 1) - df['first_active_month'].dt.date).dt.days
    return df

In [67]:
def aggregate_transactions(history):
    history.loc[:, 'purchase_date'] = pd.DatetimeIndex(history['purchase_date']).astype(np.int64) * 1e-9
    print(history.head())
    agg_func = {
        'category_1': ['sum', 'mean'],
        'category_2_1.0': ['mean'],
        'category_2_2.0': ['mean'],
        'category_2_3.0': ['mean'],
        'category_2_4.0': ['mean'],
        'category_2_5.0': ['mean'],
        'category_3_A': ['mean'],
        'category_3_B': ['mean'],
        'category_3_C': ['mean'],
        'merchant_id': ['nunique'],
        'merchant_category_id': ['nunique'],
        'state_id': ['nunique'],
        'city_id': ['nunique'],
        'subsector_id': ['nunique'],
        'purchase_amount': ['sum', 'mean', 'max', 'min', 'std'],
        'installments': ['sum', 'mean', 'max', 'min', 'std'],
        'purchase_month': ['mean', 'max', 'min', 'std'],
        'purchase_date': [np.ptp, 'min', 'max'],
        'month_lag': ['mean', 'max', 'min', 'std'],
        'month_diff': ['mean']
    }
    agg_history = history.groupby(['card_id']).agg(agg_func)
    print("\nagg_history:\n", agg_history.head())
    agg_history.columns = ['_'.join(col).strip() for col in agg_history.columns.values]
    print("\nagg_history:\n", agg_history.head())
    agg_history.reset_index(inplace=True)
    print("\nagg_history:\n", agg_history.head())
    df = (history.groupby('card_id').size().reset_index(inplace=True))
    print("\ndf:\n", agg_history.head())
    agg_history = pd.merge(df, agg_history, on='card_id', how='left')
    print("\nagg_history:\n", agg_history.head())
    return agg_history

In [68]:
def aggregate_per_month(history):
    grouped = history.groupby(['card_id', 'month_lag'])
    
    agg_func = {
        'purchase_amount': ['count', 'sum', 'mean', 'min', 'max', 'std'],
        'installments': ['count', 'sum', 'mean', 'min', 'max', 'std'],
    }
    
    intermediate_group = grouped.agg(agg_func)
    intermediate_group = ['_'.join(col).strip() for col in intermediate_group.columns.values]
    intermediate_group.reset_index(inplace=True)
    
    final_group = intermediate_group.groupby('card_id').agg(['mean', 'std'])
    final_group = ['_'.join(col).strip() for col in final_group.columns.values]
    final_group.reset_index(inplace=True)
    
    return final_group

In [6]:
new_transactions_df = pd.read_csv('../input/new_merchant_transactions.csv', parse_dates=['purchase_date'])

In [7]:
historical_transactions_df = pd.read_csv('../input/historical_transactions.csv', parse_dates=['purchase_date'])

In [8]:
new_transactions_df.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_415bb3a509,107,N,1,B,307,M_ID_b0c793002c,1,-0.557574,2018-03-11 14:57:36,1.0,9,19
1,Y,C_ID_415bb3a509,140,N,1,B,307,M_ID_88920c89e8,1,-0.569580,2018-03-19 18:53:37,1.0,9,19
2,Y,C_ID_415bb3a509,330,N,1,B,507,M_ID_ad5237ef6b,2,-0.551037,2018-04-26 14:08:44,1.0,9,14
3,Y,C_ID_415bb3a509,-1,Y,1,B,661,M_ID_9e84cda3b1,1,-0.671925,2018-03-07 09:43:21,NaN,-1,8
4,Y,C_ID_ef55cf8d4b,-1,Y,1,B,166,M_ID_3c86fa3831,1,-0.659904,2018-03-22 21:07:53,NaN,-1,29


In [9]:
historical_transactions_df.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37
3,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34
4,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37


In [10]:
new_transactions_df = binarize(new_transactions_df)

In [11]:
historical_transactions_df = binarize(historical_transactions_df)

In [19]:
print(historical_transactions_df[historical_transactions_df['authorized_flag'] == 1].shape[0])
print(historical_transactions_df[historical_transactions_df['authorized_flag'] == 0].shape[0])
print(historical_transactions_df[historical_transactions_df['category_1'] == 1].shape[0])
print(historical_transactions_df[historical_transactions_df['category_1'] == 0].shape[0])

26595452
2516909
2084029
27028332


In [23]:
train = read_data('../input/train.csv')

In [24]:
test = read_data('../input/test.csv')

In [25]:
target = train['target']
del train['target']

In [26]:
train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,elapsed_time
0,2017-06-01,C_ID_92a2005557,5,2,1,245
1,2017-01-01,C_ID_3d0044924f,4,1,0,396
2,2016-08-01,C_ID_d639edf6cd,2,2,0,549
3,2017-09-01,C_ID_186d6a6901,4,3,0,153
4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,92


In [27]:
historical_transactions_df['month_diff'] = (
    (datetime.datetime.today() - historical_transactions_df['purchase_date']).dt.days
) // 30
historical_transactions_df['month_diff'] += historical_transactions_df['month_lag']

new_transactions_df['month_diff'] = ((datetime.datetime.today() - new_transactions_df['purchase_date']).dt.days) // 30
new_transactions_df['month_diff'] += new_transactions_df['month_lag']

In [28]:
historical_transactions_df.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,month_diff
0,1,C_ID_4e6213e9bc,88,0,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37,11
1,1,C_ID_4e6213e9bc,88,0,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16,11
2,1,C_ID_4e6213e9bc,88,0,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37,12
3,1,C_ID_4e6213e9bc,88,0,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34,12
4,1,C_ID_4e6213e9bc,88,0,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37,12


In [30]:
historical_transactions_df = pd.get_dummies(historical_transactions_df, columns=['category_2', 'category_3'])

In [31]:
new_transactions_df = pd.get_dummies(new_transactions_df, columns=['category_2', 'category_3'])

In [32]:
historical_transactions_df.head()

,authorized_flag,card_id,city_id,category_1,installments,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,state_id,subsector_id,month_diff,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,category_3_A,category_3_B,category_3_C
0,1,C_ID_4e6213e9bc,88,0,0,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,16,37,11,1,0,0,0,0,1,0,0
1,1,C_ID_4e6213e9bc,88,0,0,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,16,16,11,1,0,0,0,0,1,0,0
2,1,C_ID_4e6213e9bc,88,0,0,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,16,37,12,1,0,0,0,0,1,0,0
3,1,C_ID_4e6213e9bc,88,0,0,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,16,34,12,1,0,0,0,0,1,0,0
4,1,C_ID_4e6213e9bc,88,0,0,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,16,37,12,1,0,0,0,0,1,0,0


In [33]:
historical_transactions_df = reduce_mem_usage(historical_transactions_df)

Mem. usage decreased to 1332.66 Mb (57.1% reduction)


In [34]:
new_transactions_df = reduce_mem_usage(new_transactions_df)

Mem. usage decreased to 86.12 Mb (58.9% reduction)


In [47]:
agg_fun = {'authorized_flag' : ('mean',)}
auth_mean = historical_transactions_df.groupby('card_id').agg(agg_fun)

In [52]:
#print(type(auth_mean))

In [53]:
#auth_mean.columns

In [54]:
#auth_mean.head()

In [50]:
auth_mean.columns = ['_'.join(col).strip() for col in auth_mean.columns.values]

In [56]:
auth_mean.head()

,authorized_flag_mean
card_id,
C_ID_00007093c1,0.765101
C_ID_0001238066,0.975610
C_ID_0001506ef0,0.939394
C_ID_0001793786,0.875000
C_ID_000183fdda,0.951389


In [57]:
auth_mean.reset_index(inplace=True)

In [58]:
auth_mean.head()

,card_id,authorized_flag_mean
0,C_ID_00007093c1,0.765101
1,C_ID_0001238066,0.975610
2,C_ID_0001506ef0,0.939394
3,C_ID_0001793786,0.875000
4,C_ID_000183fdda,0.951389


In [59]:
auth_mean.index

RangeIndex(start=0, stop=325540, step=1)

In [60]:
auth_mean.columns

Index(['card_id', 'authorized_flag_mean'], dtype='object')

In [61]:
authorized_transactions_df = historical_transactions_df[historical_transactions_df['authorized_flag'] == 1]
historical_transactions_df = historical_transactions_df[historical_transactions_df['authorized_flag'] == 0]

In [64]:
print(authorized_transactions_df.shape)
print(historical_transactions_df.shape)

(26595452, 21)
(2516909, 21)


In [66]:
historical_transactions_df['purchase_month'] = historical_transactions_df['purchase_date'].dt.month
authorized_transactions_df['purchase_month'] = authorized_transactions_df['purchase_date'].dt.month
new_transactions_df['purchase_date'] = new_transactions_df['purchase_date'].dt.month